In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import xgboost as xgb

In [2]:
tf.random.set_seed(42)

# DATA IMPORT

In [3]:
X = pd.read_csv(r"ProcessedFeatures.csv")
y = pd.read_csv(r"ProcessedLabels.csv")

In [4]:
zero_day_labels = ['Web Attack - XSS', 'Heartbleed', 'Web Attack - SQL Injection']
df = X.copy()
df["Label"] = y["Label"]

In [5]:
test_attacks_df = df[df["Label"].isin(zero_day_labels)].copy()
benign_df = df[df["Label"] == "BENIGN"]
test_benign_df = benign_df.sample(frac=0.1, random_state=42)  # 10% of benign data
test_df = pd.concat([test_attacks_df, test_benign_df], ignore_index=True)
train_df = df.drop(index=test_df.index)
test_df.to_csv("test_data_for_simulation.csv", index=False)

In [6]:
X_train = train_df.drop(columns=["Label"])
y_train_raw = train_df["Label"]
y_train = y_train_raw.apply(lambda x: 0 if x == "BENIGN" else 1)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [8]:
X_benign_scaled = X_train_scaled[y_train == 0]

# AUTOENCODER

In [9]:
input_dim = X_train.shape[1]
encoding_dim = 16

# Encoder
input_layer = Input(shape=(input_dim,))
encoded = layers.Dense(64, activation="relu")(input_layer)
encoded = layers.Dense(encoding_dim, activation="relu")(encoded)

# Decoder
decoded = layers.Dense(64, activation="relu")(encoded)
decoded = layers.Dense(input_dim, activation="linear")(decoded)

autoencoder = Model(input_layer, decoded, name="autoencoder")
encoder = Model(input_layer, encoded, name="encoder")

In [10]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_benign_scaled, X_benign_scaled, epochs=50, batch_size=64, verbose=1)

Epoch 1/50
22340/22340 [==============================] - 50s 2ms/step - loss: 0.1100
Epoch 2/50
22340/22340 [==============================] - 41s 2ms/step - loss: 0.0594
Epoch 3/50
22340/22340 [==============================] - 49s 2ms/step - loss: 0.0420
Epoch 4/50
22340/22340 [==============================] - 35s 2ms/step - loss: 0.0453
Epoch 5/50
22340/22340 [==============================] - 33s 1ms/step - loss: 0.0397
Epoch 6/50
22340/22340 [==============================] - 48s 2ms/step - loss: 0.0289
Epoch 7/50
22340/22340 [==============================] - 40s 2ms/step - loss: 0.0330
Epoch 8/50
22340/22340 [==============================] - 41s 2ms/step - loss: 0.0245
Epoch 9/50
22340/22340 [==============================] - 35s 2ms/step - loss: 0.0264
Epoch 10/50
22340/22340 [==============================] - 31s 1ms/step - loss: 0.0256
Epoch 11/50
22340/22340 [==============================] - 32s 1ms/step - loss: 0.0234
Epoch 12/50
22340/22340 [===========================

In [11]:
autoencoder.save("autoencoder_model.keras")
encoder.save("ae_encoder.keras")

# XGBoost Downstream

In [12]:
z_train = encoder.predict(X_train_scaled)
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric="logloss")

60896/60896 [==============================] - 64s 1ms/step


In [13]:
xgb_model.fit(z_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [14]:
xgb_model.save_model("xgb_model.json")